In [2]:
import pandas as pd

In [3]:
ds = pd.read_csv("Social_Network_Ads.csv")

In [4]:
ds=pd.get_dummies(ds,drop_first=True)

In [5]:
ds.drop("User ID",axis=1) #axis 1 Column, 0 Row

,Age,EstimatedSalary,Purchased,Gender_Male
0,19,19000,0,True
1,35,20000,0,True
2,26,43000,0,False
3,27,57000,0,False
4,19,76000,0,True
...,...,...,...,...
395,46,41000,1,False
396,51,23000,1,True
397,50,20000,1,False
398,36,33000,0,True


In [6]:
ds['Purchased'].value_counts() # to check balance or inbalanced data set - inbalanced 

Purchased
0    257
1    143
Name: count, dtype: int64

In [7]:
independent = ds[['Age', 'EstimatedSalary','Gender_Male']]
dependent = ds[['Purchased']]

In [8]:
from sklearn.model_selection import train_test_split
x_train,x_test,y_train,y_test=train_test_split(independent,dependent,test_size=1/3,random_state=False)

In [36]:
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import GridSearchCV
param_grid = {
        'C': [0.001, 0.01, 0.1, 1, 10, 100],
        'penalty': ['l1', 'l2'],
        'solver': ['liblinear', 'lbfgs', 'newton-cg', 'saga'] # 'liblinear' supports both 'l1' and 'l2' penalties
    }
'''
param_grid = {
        'C': [0.001, 0.01, 0.1, 1, 10, 100],
        'penalty': ['l2'],
        'solver': ['liblinear', 'lbfgs', 'newton-cg', 'saga'] # 'liblinear' supports both 'l1' and 'l2' penalties
    }
''' 
# prefer F1_weighted as it suits all case like imbalanced balanced and precision and recall method
#Classifier = GridSearchCV(LogisticRegression(),param_grid, refit = True, scoring='f1_weighted', n_jobs=-1, verbose = 0) # Scoring - accuracy, precision, recall, macro avaerage, weighted average, f1_weighted
#verbose = 3, 3 or higher	Very detailed info (step-by-step),  0	No output (silent mode), 1	Basic info (minimal messages), 2	More info (intermediate messages)
Classifier = GridSearchCV(LogisticRegression(),param_grid, refit = True, scoring='roc_auc', n_jobs=-1, verbose = 0) 
# Creats the model
Classifier.fit(x_train,y_train)

C:\Users\hitov\anaconda3\Lib\site-packages\sklearn\model_selection\_validation.py:540: FitFailedWarning: 
60 fits failed out of a total of 240.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
30 fits failed with the following error:
Traceback (most recent call last):
  File "C:\Users\hitov\anaconda3\Lib\site-packages\sklearn\model_selection\_validation.py", line 888, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "C:\Users\hitov\anaconda3\Lib\site-packages\sklearn\base.py", line 1473, in wrapper
    return fit_method(estimator, *args, **kwargs)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "C:\Users\hitov\anaconda3\Lib\site-packages\sklearn\linear_model\_logistic.py", line 1194, in fit
    solver = 

GridSearchCV(estimator=LogisticRegression(), n_jobs=-1,
             param_grid={'C': [0.001, 0.01, 0.1, 1, 10, 100],
                         'penalty': ['l1', 'l2'],
                         'solver': ['liblinear', 'lbfgs', 'newton-cg', 'saga']},
             scoring='roc_auc')

In [37]:
result = Classifier.cv_results_

In [38]:
y_pred = Classifier.predict(x_test)

In [39]:
# Built the matrix
from sklearn.metrics import confusion_matrix
matrix = confusion_matrix(y_test,y_pred)
print(matrix)

[[79  6]
 [12 37]]


In [40]:
# fetch the Classification report
from sklearn.metrics import classification_report
report = classification_report(y_test,y_pred)
print(report)

              precision    recall  f1-score   support

           0       0.87      0.93      0.90        85
           1       0.86      0.76      0.80        49

    accuracy                           0.87       134
   macro avg       0.86      0.84      0.85       134
weighted avg       0.87      0.87      0.86       134



In [41]:
# Key to built the model
from sklearn.metrics import f1_score
f1_macro = f1_score (y_test,y_pred,average='weighted')
print ("The best value for Parameter {}:".format(Classifier.best_params_),f1_macro)


The best value for Parameter {'C': 0.01, 'penalty': 'l2', 'solver': 'newton-cg'}: 0.8635810571647691


In [44]:
from sklearn.metrics import roc_auc_score
roc_auc = roc_auc_score (y_test,Classifier.predict_proba(x_test)[:,1]) # : tell all rows, take 1 ist colum of all rows for probability 
print ("The best value for Parameter {}:".format(Classifier.best_params_),roc_auc)
  

The best value for Parameter {'C': 0.01, 'penalty': 'l2', 'solver': 'newton-cg'}: 0.9474189675870348


In [46]:
table = pd.DataFrame.from_dict(result)
table

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_C,param_penalty,param_solver,params,split0_test_score,split1_test_score,split2_test_score,split3_test_score,split4_test_score,mean_test_score,std_test_score,rank_test_score
0,0.035828,0.000502,0.013309,0.000908,0.001,l1,liblinear,"{'C': 0.001, 'penalty': 'l1', 'solver': 'libli...",0.281955,0.334921,0.429567,0.222910,0.341331,0.322137,0.068605,17
1,0.014198,0.000215,0.000000,0.000000,0.001,l1,lbfgs,"{'C': 0.001, 'penalty': 'l1', 'solver': 'lbfgs'}",NaN,NaN,NaN,NaN,NaN,NaN,NaN,37
2,0.006257,0.006875,0.000000,0.000000,0.001,l1,newton-cg,"{'C': 0.001, 'penalty': 'l1', 'solver': 'newto...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,37
3,0.011638,0.003519,0.007821,0.001298,0.001,l1,saga,"{'C': 0.001, 'penalty': 'l1', 'solver': 'saga'}",0.281955,0.334921,0.429567,0.222910,0.341331,0.322137,0.068605,17
4,0.009370,0.001396,0.007613,0.001832,0.001,l2,liblinear,"{'C': 0.001, 'penalty': 'l2', 'solver': 'libli...",0.090226,0.139683,0.244582,0.148607,0.113003,0.147220,0.052834,36
5,0.051728,0.006906,0.012124,0.005146,0.001,l2,lbfgs,"{'C': 0.001, 'penalty': 'l2', 'solver': 'lbfgs'}",0.914286,0.877778,0.817337,0.945820,0.978328,0.906710,0.055751,14
6,0.109746,0.015078,0.006429,0.001883,0.001,l2,newton-cg,"{'C': 0.001, 'penalty': 'l2', 'solver': 'newto...",0.914286,0.877778,0.817337,0.945820,0.978328,0.906710,0.055751,14
7,0.020947,0.008251,0.008432,0.005442,0.001,l2,saga,"{'C': 0.001, 'penalty': 'l2', 'solver': 'saga'}",0.279699,0.333333,0.427245,0.221362,0.340557,0.320439,0.068490,20
8,0.005381,0.002809,0.006512,0.000498,0.010,l1,liblinear,"{'C': 0.01, 'penalty': 'l1', 'solver': 'liblin...",0.273684,0.333333,0.377709,0.222910,0.289474,0.299422,0.052715,30
9,0.000202,0.000404,0.000000,0.000000,0.010,l1,lbfgs,"{'C': 0.01, 'penalty': 'l1', 'solver': 'lbfgs'}",NaN,NaN,NaN,NaN,NaN,NaN,NaN,37


In [ ]:
age_input=float(input("Age:"))
salary_input=float(input("Salary:"))
sex_male_input=int(input("Sex Male 0 or 1:"))

In [ ]:
Classifier.predict([[age_input,salary_input,sex_male_input]])